In [1]:
using Printf
using Dates
using RandomNumbers
using StaticArrays
using LinearAlgebra
using Flux
using Plots
using Statistics

include("src/distances.jl");

# 1. Input configuration

`Use ; sign to run OS commands:`

In [2]:
;ls

algorithms
LICENSE
LJ-data.ipynb
LJ-init.in
LJML-init.in
LJ-test.in
mcLJ-run.jl
mctraj-p001.xyz
ML-IMC.ipynb
rdf-mean-p40.dat
README.md
src


This is the function that allows to read XYZ output files. I will use the coordinates as an input data for the training.

In [3]:
"""
readXYZ(xyzname)

Reads an XYZ file and outputs coordinates of all atoms
arranged in separate frames
"""
function readXYZ(xyzname)
    file = open(xyzname, "r")
    lines = readlines(file)
    natoms = parse(Int, lines[1])
    ncomments = 2
    nlines = length(lines)
    nframes = Int((nlines % natoms) / 2)
    xyz = [[zeros(3) for i in 1:natoms] for j in 1:nframes]
    println("Reading $(xyzname) with $(nlines) lines...")
    println("Found $(nframes) frames with $(natoms) atoms each...")
    for lineId in 1:nlines
        frameId = ceil(Int, lineId/(natoms + ncomments))
        atomId = lineId - ((frameId - 1) * (natoms + ncomments)) - ncomments
        @assert atomId <= natoms
        if atomId > 0
            atomLine = split(lines[lineId])
            for i in 1:3
                xyz[frameId][atomId][i] = parse(Float64, atomLine[i+1])
            end
        end
    end
    close(file)
    println("Success! Closing the file...")
    return(xyz)
end;

Reading the configurations:

In [4]:
xyz = readXYZ("mctraj-p001.xyz");

Reading mctraj-p001.xyz with 6012 lines...
Found 6 frames with 1000 atoms each...
Success! Closing the file...


Saving the final configuration:

In [5]:
confout = xyz[end];

# 2. Reference distance histogram

In [6]:
"""
readRDF(rdfname)

Reads RDF and distance histogram produced
by mcLJ.jl
"""
function readRDF(rdfname)
    file = open(rdfname, "r")
    lines = readlines(file)
    ncomments = 2
    nlines = length(lines) - ncomments
    bins = zeros(nlines)
    rdf = zeros(nlines)
    hist = zeros(nlines)
    for i in (1 + ncomments):length(lines)
        rdfline = split(lines[i])
        if length(rdfline) == 3
            bins[i - ncomments] = parse(Float64, rdfline[1])
            rdf[i - ncomments] = parse(Float64, rdfline[2])
            hist[i - ncomments] = parse(Float64, rdfline[3])
        end
    end
    return(bins, rdf, hist)
end;

In [7]:
bins, refrdf, refhist = readRDF("rdf-mean-p40.dat");

# 3. ML-IMC input parameters

In [69]:
"""
struct inputParms

Fields:
box: box vector, Å
T: temperature, K
Δ: max displacement, Å
steps: total number of steps
Eqsteps: equilibration steps
xyzout: XYZ output frequency
outfreq: output frequency
"""
struct inputParms
    box::SVector{3, Float64}
    T::Float64
    Δ::Float64  
    steps::Int
    Eqsteps::Int
    xyzout::Int 
    outfreq::Int
end

"""
readinput(inputname)

Reads an input file for ML-IMC
and saves the data into the
inputParms struct
"""
function readinput(inputname)
    # Has to define the variable outside of the main loop
    box = zeros(3)
    T::Float64 = 0.
    Δ::Float64 = 0.
    steps::Int = 0
    Eqsteps::Int = 0
    xyzout::Int = 0
    outfreq::Int = 0
    file = open(inputname, "r")
    lines = readlines(file)
    for line in lines
        if length(line) > 0 && line[1] != '#'
            splittedLine = split(line)
            if splittedLine[1] == "box"
                box[1] = parse(Float64, splittedLine[3])
                box[2] = parse(Float64, splittedLine[4])
                box[3] = parse(Float64, splittedLine[5])
            elseif splittedLine[1] == "temperature"
                T = parse(Float64, splittedLine[3])
            elseif splittedLine[1] == "delta"
                Δ = parse(Float64, splittedLine[3])
            elseif splittedLine[1] == "steps"
                steps = Int(parse(Float64, splittedLine[3]))
            elseif splittedLine[1] == "Eqsteps"
                Eqsteps = Int(parse(Float64, splittedLine[3]))
            elseif splittedLine[1] == "xyzout"
                xyzout = Int(parse(Float64, splittedLine[3]))
            elseif splittedLine[1] == "outfreq"
                outfreq = Int(parse(Float64, splittedLine[3]))
            end
        end
    end
    # Save parameters into the inputParms struct
    parameters = inputParms(box, T, Δ, steps, Eqsteps, xyzout, outfreq)
    return(parameters)
end;

In [70]:
parameters = readinput("LJML-init.in");

In [71]:
parameters.box

3-element SVector{3, Float64} with indices SOneTo(3):
 36.405
 36.405
 36.405